## ⚠️This notebook is FORKED from [this](https://www.kaggle.com/code/hyperc/gsdc-reproducing-baseline-wls-on-one-measurement).

In [ ]:
import glob
from dataclasses import dataclass
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline
import scipy.optimize as opt
pd.set_option('display.max_columns',100)
INPUT_PATH = '../input/smartphone-decimeter-2022'

WGS84_SEMI_MAJOR_AXIS = 6378137.0
WGS84_SEMI_MINOR_AXIS = 6356752.314245
WGS84_SQUARED_FIRST_ECCENTRICITY  = 6.69437999013e-3
WGS84_SQUARED_SECOND_ECCENTRICITY = 6.73949674226e-3

HAVERSINE_RADIUS = 6_371_000


In [ ]:
@dataclass
class ECEF:
    x: np.array
    y: np.array
    z: np.array

    def to_numpy(self):
        return np.stack([self.x, self.y, self.z], axis=0)

    @staticmethod
    def from_numpy(pos):
        x, y, z = [np.squeeze(w) for w in np.split(pos, 3, axis=-1)]
        return ECEF(x=x, y=y, z=z)

@dataclass
class BLH:
    lat : np.array
    lng : np.array
    hgt : np.array

def ECEF_to_BLH(ecef):
    a = WGS84_SEMI_MAJOR_AXIS
    b = WGS84_SEMI_MINOR_AXIS
    e2  = WGS84_SQUARED_FIRST_ECCENTRICITY
    e2_ = WGS84_SQUARED_SECOND_ECCENTRICITY
    x = ecef.x
    y = ecef.y
    z = ecef.z
    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(z * (a/b), r)
    B = np.arctan2(z + (e2_*b)*np.sin(t)**3, r - (e2*a)*np.cos(t)**3)
    L = np.arctan2(y, x)
    n = a / np.sqrt(1 - e2*np.sin(B)**2)
    H = (r / np.cos(B)) - n
    return BLH(lat=B, lng=L, hgt=H)

def haversine_distance(blh_1, blh_2):
    dlat = blh_2.lat - blh_1.lat
    dlng = blh_2.lng - blh_1.lng
    a = np.sin(dlat/2)**2 + np.cos(blh_1.lat) * np.cos(blh_2.lat) * np.sin(dlng/2)**2
    dist = 2 * HAVERSINE_RADIUS * np.arcsin(np.sqrt(a))
    return dist

def pandas_haversine_distance(df1, df2):
    blh1 = BLH(
        lat=np.deg2rad(df1['LatitudeDegrees'].to_numpy()),
        lng=np.deg2rad(df1['LongitudeDegrees'].to_numpy()),
        hgt=0,
    )
    blh2 = BLH(
        lat=np.deg2rad(df2['LatitudeDegrees'].to_numpy()),
        lng=np.deg2rad(df2['LongitudeDegrees'].to_numpy()),
        hgt=0,
    )
    return haversine_distance(blh1, blh2)

# Apply WLS on one collection and one measurement

In [ ]:
trip_id = '2020-05-15-US-MTV-1/GooglePixel4XL'
df = pd.read_csv(f'../input/smartphone-decimeter-2022/train/{trip_id}/device_gnss.csv')

In [ ]:
measurement_epoch_time = 1589573679445
df = df[df.utcTimeMillis == measurement_epoch_time]
df

In [ ]:
# Corrected pseudorange according to data instructions
df['correctedPrM'] = df.apply(
    lambda r: r.RawPseudorangeMeters + r.SvClockBiasMeters - r.IsrbMeters - r.IonosphericDelayMeters - r.TroposphericDelayMeters,
    axis=1
)

# Time it took for signal to travel
light_speed = 299_792_458
df['transmissionTimeSeconds'] = df['correctedPrM'] / light_speed

In [ ]:
# Compute true sat positions at arrival time
omega_e = 7.2921151467e-5
df['xSatPosMRotated'] = \
    np.cos(omega_e * df['transmissionTimeSeconds']) * df['SvPositionXEcefMeters'] \
    + np.sin(omega_e * df['transmissionTimeSeconds']) * df['SvPositionYEcefMeters']
    
df['ySatPosMRotated'] = \
    - np.sin(omega_e * df['transmissionTimeSeconds']) * df['SvPositionXEcefMeters'] \
    + np.cos(omega_e * df['transmissionTimeSeconds']) * df['SvPositionYEcefMeters']
    
df['zSatPosMRotated'] = df['SvPositionZEcefMeters']

In [ ]:
# Uncertainty weight for the WLS method
df['uncertaintyWeight'] = 1 / df['RawPseudorangeUncertaintyMeters']

In [ ]:
# Set up least squares methods
def distance(sat_pos, x):
    sat_pos_diff = sat_pos.copy(deep=True)
    
    sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
    sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
    sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

    sat_pos_diff['d'] = sat_pos_diff.apply(
        lambda r: r.uncertaintyWeight * 
            (np.sqrt((r.xSatPosMRotated**2 + r.ySatPosMRotated**2 + r.zSatPosMRotated**2)) + x[3] - r.correctedPrM),
        axis=1
    )

    return sat_pos_diff['d']

def distance_fixed_satpos(x):
    return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)

In [ ]:
# Start point for the optimiser
x0= [0,0,0,0]

opt_res = opt.least_squares(distance_fixed_satpos, x0)

# Optimiser yields a position in the ECEF coordinates
opt_res_pos = opt_res.x

In [ ]:
wls_estimated_pos = ECEF_to_BLH(ECEF.from_numpy(opt_res_pos[:3]))

In [ ]:
baseline = df[['WlsPositionXEcefMeters','WlsPositionYEcefMeters','WlsPositionZEcefMeters']].values[0]
baseline = ECEF_to_BLH(ECEF.from_numpy(baseline))

In [ ]:
gt_df = pd.read_csv(f'../input/smartphone-decimeter-2022/train/{trip_id}/ground_truth.csv')
gt_df = gt_df[gt_df.UnixTimeMillis==measurement_epoch_time]
gt = gt_df[['LatitudeDegrees','LongitudeDegrees']].values[0]
gt = BLH(*np.deg2rad(gt),None)

In [ ]:
print("Baseline distance with groundtruth position (m)")
haversine_distance(baseline,gt)

In [ ]:
print("Our estimated position (with WLS) distance with groundtruth position (m)")
haversine_distance(wls_estimated_pos,gt)